**Import the necessary libraries**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc, plot_precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

**Load data into pandas dataframe**


In [2]:
user_data = pd.read_csv(r'D:\MACHINE LEARNING\Mentorship program\ML Project1\features_data.csv')
equity_data = pd.read_csv(r'D:\MACHINE LEARNING\Mentorship program\ML Project1\equity_value_data.csv')

In [3]:
user_data.head()

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91


In [4]:
equity_data.head()

,timestamp,close_equity,user_id
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03


In [5]:
equity_data_first = equity_data.groupby("user_id").first()

In [6]:
equity_data_first=equity_data_first.reset_index('user_id')

In [7]:
equity_data_first

,user_id,timestamp,close_equity
0,0012db34aa7b083f5714e7831195e54d,2016-08-18T00:00:00Z,1211.6055
1,001d6c77dbdb3213cead7673f250bfdc,2016-08-18T00:00:00Z,2756.3100
2,002e4653171ddc61c3cd30603cd7bd3e,2017-02-16T00:00:00Z,100.0000
3,00384fa9be6fdca1b786bae70d78f88f,2017-02-23T00:00:00Z,505.9700
4,0042aac295a0d4df88f4b83012778bd4,2016-08-18T00:00:00Z,131.5515
...,...,...,...
5579,ff9ee08791e20724a86995ab2bc72578,2016-08-18T00:00:00Z,226.8878
5580,ffa12d2f97e310910291f9b26fb2318d,2016-08-17T00:00:00Z,2750.9388
5581,ffae713096867a32e74f633060667153,2016-09-28T00:00:00Z,19.7401
5582,ffbda9a14e07718e2b21fb03896d21c5,2017-02-15T00:00:00Z,42.7200


**To check if any of the users started before 2016-08-16**

In [51]:
import datetime
date1=datetime.date(2016,8,16)
equity_data_first['timestamp']=pd.to_datetime(equity_data_first['timestamp']).dt.date
# equity_data_first[equity_data_first['timestamp'].iloc[:]>=date1]
equity_data_first['tstmp']=equity_data_first['timestamp']-date1
equity_data_first['days_diff']=equity_data_first['tstmp'].dt.days
equity_data_first

,user_id,timestamp,close_equity,tstmp,days_diff
0,0012db34aa7b083f5714e7831195e54d,2016-08-18,1211.6055,2 days,2
1,001d6c77dbdb3213cead7673f250bfdc,2016-08-18,2756.3100,2 days,2
2,002e4653171ddc61c3cd30603cd7bd3e,2017-02-16,100.0000,184 days,184
3,00384fa9be6fdca1b786bae70d78f88f,2017-02-23,505.9700,191 days,191
4,0042aac295a0d4df88f4b83012778bd4,2016-08-18,131.5515,2 days,2
...,...,...,...,...,...
5579,ff9ee08791e20724a86995ab2bc72578,2016-08-18,226.8878,2 days,2
5580,ffa12d2f97e310910291f9b26fb2318d,2016-08-17,2750.9388,1 days,1
5581,ffae713096867a32e74f633060667153,2016-09-28,19.7401,43 days,43
5582,ffbda9a14e07718e2b21fb03896d21c5,2017-02-15,42.7200,183 days,183


In [10]:
equity_data_last = equity_data.groupby("user_id").last()
equity_data_last=equity_data_last.reset_index('user_id')
equity_data_last

,user_id,timestamp,close_equity
0,0012db34aa7b083f5714e7831195e54d,2017-08-17T00:00:00Z,487.7885
1,001d6c77dbdb3213cead7673f250bfdc,2017-08-17T00:00:00Z,3981.3350
2,002e4653171ddc61c3cd30603cd7bd3e,2017-08-18T00:00:00Z,134.3800
3,00384fa9be6fdca1b786bae70d78f88f,2017-08-18T00:00:00Z,426.2400
4,0042aac295a0d4df88f4b83012778bd4,2017-08-17T00:00:00Z,1769.2150
...,...,...,...
5579,ff9ee08791e20724a86995ab2bc72578,2017-08-17T00:00:00Z,62.1951
5580,ffa12d2f97e310910291f9b26fb2318d,2017-08-16T00:00:00Z,7448.1895
5581,ffae713096867a32e74f633060667153,2017-08-17T00:00:00Z,32.9300
5582,ffbda9a14e07718e2b21fb03896d21c5,2017-08-18T00:00:00Z,25.5900


**To check if the user is churned based on the difference between the last collection date(2017-08-18) and the last entry date of the user**

In [11]:
import datetime
date2=datetime.date(2017,8,18)
equity_data_last['timestamp']=pd.to_datetime(equity_data_last['timestamp']).dt.date
equity_data_last['tstmp']=date2-equity_data_last['timestamp']
equity_data_last['days_diff']=equity_data_last['tstmp'].dt.days
equity_data_last[equity_data_last['days_diff']>=28]

,user_id,timestamp,close_equity,tstmp,days_diff
6,004aab1640f3a04b87b1f404fb4c018d,2017-04-28,25.1400,112 days,112
7,004ea9d7662aa8dc840bbff212cfa4b8,2017-07-10,100.9800,39 days,39
15,00f89f56f25989b0bb7ea05bac2dccc4,2017-05-19,1143.6600,91 days,91
34,01ef51aa8fab66b13d2249bc6ee68975,2017-05-11,27.9182,99 days,99
35,02012c83f5c4508404312aec848e2deb,2017-05-15,1773.7500,95 days,95
...,...,...,...,...,...
5538,fdbb2b7567e059cd920c8401906caa6d,2017-04-21,55.3200,119 days,119
5543,fe17d44323f4bbf332cc2432356116b8,2017-06-09,13.3100,70 days,70
5552,fe8eed9b8cdc7e429d311fffdfa7c9fe,2017-06-22,5536.9400,57 days,57
5562,ff05ebb811f35d2ad064e9026265c4bc,2017-01-19,17.8700,211 days,211


In [12]:
equity_data_last['churned_last']=np.where(equity_data_last['days_diff']>=28,'yes','no')
equity_data_last

,user_id,timestamp,close_equity,tstmp,days_diff,churned_last
0,0012db34aa7b083f5714e7831195e54d,2017-08-17,487.7885,1 days,1,no
1,001d6c77dbdb3213cead7673f250bfdc,2017-08-17,3981.3350,1 days,1,no
2,002e4653171ddc61c3cd30603cd7bd3e,2017-08-18,134.3800,0 days,0,no
3,00384fa9be6fdca1b786bae70d78f88f,2017-08-18,426.2400,0 days,0,no
4,0042aac295a0d4df88f4b83012778bd4,2017-08-17,1769.2150,1 days,1,no
...,...,...,...,...,...,...
5579,ff9ee08791e20724a86995ab2bc72578,2017-08-17,62.1951,1 days,1,no
5580,ffa12d2f97e310910291f9b26fb2318d,2017-08-16,7448.1895,2 days,2,no
5581,ffae713096867a32e74f633060667153,2017-08-17,32.9300,1 days,1,no
5582,ffbda9a14e07718e2b21fb03896d21c5,2017-08-18,25.5900,0 days,0,no


**To calculate the difference between consecutive dates for each user separately in gap column**

In [13]:
equity_data_new =equity_data.sort_values(by=['user_id', 'timestamp']).reset_index(drop=True)
equity_data_new['timestamp']=pd.to_datetime(equity_data_new['timestamp']).dt.date
equity_data_new['gap'] = equity_data_new.groupby('user_id')['timestamp'].diff()
equity_data_new

,timestamp,close_equity,user_id,gap
0,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d,NaT
1,2016-08-19,1173.5640,0012db34aa7b083f5714e7831195e54d,1 days 00:00:00
2,2016-08-22,1253.0597,0012db34aa7b083f5714e7831195e54d,3 days 00:00:00
3,2016-08-23,1252.9050,0012db34aa7b083f5714e7831195e54d,1 days 00:00:00
4,2016-08-24,1262.1360,0012db34aa7b083f5714e7831195e54d,1 days 00:00:00
...,...,...,...,...
1119153,2017-08-14,2156.2400,ffc1e622f3a0b2666f09a6dcb7f27918,3 days 00:00:00
1119154,2017-08-15,2134.7100,ffc1e622f3a0b2666f09a6dcb7f27918,1 days 00:00:00
1119155,2017-08-16,2152.1200,ffc1e622f3a0b2666f09a6dcb7f27918,1 days 00:00:00
1119156,2017-08-17,2042.2800,ffc1e622f3a0b2666f09a6dcb7f27918,1 days 00:00:00


**To check for Null valules in the data in gap column**

In [14]:
equity_data_new[equity_data_new.isin([np.nan,pd.NaT]).any(1)]

,timestamp,close_equity,user_id,gap
0,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d,NaT
252,2016-08-18,2756.3100,001d6c77dbdb3213cead7673f250bfdc,NaT
504,2017-02-16,100.0000,002e4653171ddc61c3cd30603cd7bd3e,NaT
632,2017-02-23,505.9700,00384fa9be6fdca1b786bae70d78f88f,NaT
756,2016-08-18,131.5515,0042aac295a0d4df88f4b83012778bd4,NaT
...,...,...,...,...
1118095,2016-08-18,226.8878,ff9ee08791e20724a86995ab2bc72578,NaT
1118347,2016-08-17,2750.9388,ffa12d2f97e310910291f9b26fb2318d,NaT
1118599,2016-09-28,19.7401,ffae713096867a32e74f633060667153,NaT
1118823,2017-02-15,42.7200,ffbda9a14e07718e2b21fb03896d21c5,NaT


**Replace the null values with timedelta (0 days)**

In [15]:
equity_data_new['gap'].fillna(pd.Timedelta('0 days'),inplace=True)

**timedelta of gap column into integer values**

In [16]:
equity_data_new['gap_int']=equity_data_new['gap'].dt.days
equity_data_new

,timestamp,close_equity,user_id,gap,gap_int
0,2016-08-18,1211.6055,0012db34aa7b083f5714e7831195e54d,0 days,0
1,2016-08-19,1173.5640,0012db34aa7b083f5714e7831195e54d,1 days,1
2,2016-08-22,1253.0597,0012db34aa7b083f5714e7831195e54d,3 days,3
3,2016-08-23,1252.9050,0012db34aa7b083f5714e7831195e54d,1 days,1
4,2016-08-24,1262.1360,0012db34aa7b083f5714e7831195e54d,1 days,1
...,...,...,...,...,...
1119153,2017-08-14,2156.2400,ffc1e622f3a0b2666f09a6dcb7f27918,3 days,3
1119154,2017-08-15,2134.7100,ffc1e622f3a0b2666f09a6dcb7f27918,1 days,1
1119155,2017-08-16,2152.1200,ffc1e622f3a0b2666f09a6dcb7f27918,1 days,1
1119156,2017-08-17,2042.2800,ffc1e622f3a0b2666f09a6dcb7f27918,1 days,1


**List of users, those are churned based on the difference between consecutive dates**

In [17]:
user_list=equity_data_new[equity_data_new['gap_int']>=28]['user_id'].drop_duplicates()
user_list=list(user_list)
churned_by_consdiff=['yes']*279
data = list(zip(user_list,churned_by_consdiff))
churned_df1 = pd.DataFrame(data, columns = ['user_id','churn/not_churn'])
churned_df1

,user_id,churn/not_churn
0,00440034cc4152bfb01b30f5c381c4e3,yes
1,005d630a68b4ab3a2f4cd49d9a87c50d,yes
2,028367ff3cbcc04c2afc2ce3336c00e2,yes
3,0423b88554cedaa7efd8dd4c81774cce,yes
4,062ea0ff3b7fc36ae471968aced1f4a1,yes
...,...,...
274,fdc54af66d1190dec81b95b4a2965634,yes
275,ff0ae95285c43e3a5af84860bffaa544,yes
276,ff377467d4e28b425266a8b2c8b2f5c7,yes
277,ff7610fdd7ac5cbfa0b17aca53af5db4,yes


**Sort the dataframe (user_data) by user_id**

In [18]:
new_user_data =user_data.sort_values(by=['user_id']).reset_index(drop=True)
new_user_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.701933,stock,100.0,long_time_horizon,0012db34aa7b083f5714e7831195e54d
1,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,19.568433,etp,1500.0,long_time_horizon,001d6c77dbdb3213cead7673f250bfdc
2,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,22.091867,stock,100.0,long_time_horizon,002e4653171ddc61c3cd30603cd7bd3e
3,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,108.555350,stock,500.0,short_time_horizon,00384fa9be6fdca1b786bae70d78f88f
4,high_risk_tolerance,good_investment_exp,very_important_liq_need,Android,39.973717,lp,100.0,short_time_horizon,0042aac295a0d4df88f4b83012778bd4
...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,4.703033,mlp,100.0,short_time_horizon,ff9ee08791e20724a86995ab2bc72578
5580,high_risk_tolerance,good_investment_exp,somewhat_important_liq_need,Android,6.988433,stock,250.0,long_time_horizon,ffa12d2f97e310910291f9b26fb2318d
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.898000,stock,20.0,med_time_horizon,ffae713096867a32e74f633060667153
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,34.150567,stock,2.0,long_time_horizon,ffbda9a14e07718e2b21fb03896d21c5


**Add churn_by_lastdate into dataframe(new_user_data)**

In [19]:
new_user_data['churn_by_lastdate'] = equity_data_last['churned_last']
new_user_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churn_by_lastdate
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.701933,stock,100.0,long_time_horizon,0012db34aa7b083f5714e7831195e54d,no
1,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,19.568433,etp,1500.0,long_time_horizon,001d6c77dbdb3213cead7673f250bfdc,no
2,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,22.091867,stock,100.0,long_time_horizon,002e4653171ddc61c3cd30603cd7bd3e,no
3,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,108.555350,stock,500.0,short_time_horizon,00384fa9be6fdca1b786bae70d78f88f,no
4,high_risk_tolerance,good_investment_exp,very_important_liq_need,Android,39.973717,lp,100.0,short_time_horizon,0042aac295a0d4df88f4b83012778bd4,no
...,...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,4.703033,mlp,100.0,short_time_horizon,ff9ee08791e20724a86995ab2bc72578,no
5580,high_risk_tolerance,good_investment_exp,somewhat_important_liq_need,Android,6.988433,stock,250.0,long_time_horizon,ffa12d2f97e310910291f9b26fb2318d,no
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.898000,stock,20.0,med_time_horizon,ffae713096867a32e74f633060667153,no
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,34.150567,stock,2.0,long_time_horizon,ffbda9a14e07718e2b21fb03896d21c5,no


**user_list1: list of users not churned in new_user_data**<br />
**Create a dataframe based on consecutive difference between dates**<br />

In [31]:
user_list1=list(set(new_user_data['user_id']).symmetric_difference(set(churned_df1['user_id'])))
len(user_list1)
churned_by_consdiff1=['no']*5305
data1= list(zip(user_list1,churned_by_consdiff1))
churned_df2 = pd.DataFrame(data1, columns = ['user_id','churn/not_churn'])
churned_df=pd.concat([churned_df1,churned_df2])
churned_df=churned_df.sort_values(by=['user_id']).reset_index(drop=True)

In [32]:
new_user_data['churn_by_consec_diff'] = churned_df['churn/not_churn']
new_user_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churn_by_lastdate,churn_by_consec_diff
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.701933,stock,100.0,long_time_horizon,0012db34aa7b083f5714e7831195e54d,no,no
1,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,19.568433,etp,1500.0,long_time_horizon,001d6c77dbdb3213cead7673f250bfdc,no,no
2,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,22.091867,stock,100.0,long_time_horizon,002e4653171ddc61c3cd30603cd7bd3e,no,no
3,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,108.555350,stock,500.0,short_time_horizon,00384fa9be6fdca1b786bae70d78f88f,no,no
4,high_risk_tolerance,good_investment_exp,very_important_liq_need,Android,39.973717,lp,100.0,short_time_horizon,0042aac295a0d4df88f4b83012778bd4,no,no
...,...,...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,4.703033,mlp,100.0,short_time_horizon,ff9ee08791e20724a86995ab2bc72578,no,no
5580,high_risk_tolerance,good_investment_exp,somewhat_important_liq_need,Android,6.988433,stock,250.0,long_time_horizon,ffa12d2f97e310910291f9b26fb2318d,no,no
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.898000,stock,20.0,med_time_horizon,ffae713096867a32e74f633060667153,no,no
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,34.150567,stock,2.0,long_time_horizon,ffbda9a14e07718e2b21fb03896d21c5,no,no


**Create a column is_churned based on both lastdates and consecutive dates**

In [34]:
final_user_churn=[]
for i in range(len(new_user_data)):
    if (new_user_data['churn_by_lastdate'][i]=='yes') or (new_user_data['churn_by_consec_diff'][i]=='yes'):
        final_user_churn.append('yes')
        i=i+1
    else:
        final_user_churn.append('no')
        i=i+1

In [35]:
new_user_data['is_churned']=final_user_churn

In [36]:
new_user_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churn_by_lastdate,churn_by_consec_diff,is_churned
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.701933,stock,100.0,long_time_horizon,0012db34aa7b083f5714e7831195e54d,no,no,no
1,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,19.568433,etp,1500.0,long_time_horizon,001d6c77dbdb3213cead7673f250bfdc,no,no,no
2,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,22.091867,stock,100.0,long_time_horizon,002e4653171ddc61c3cd30603cd7bd3e,no,no,no
3,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,108.555350,stock,500.0,short_time_horizon,00384fa9be6fdca1b786bae70d78f88f,no,no,no
4,high_risk_tolerance,good_investment_exp,very_important_liq_need,Android,39.973717,lp,100.0,short_time_horizon,0042aac295a0d4df88f4b83012778bd4,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,4.703033,mlp,100.0,short_time_horizon,ff9ee08791e20724a86995ab2bc72578,no,no,no
5580,high_risk_tolerance,good_investment_exp,somewhat_important_liq_need,Android,6.988433,stock,250.0,long_time_horizon,ffa12d2f97e310910291f9b26fb2318d,no,no,no
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.898000,stock,20.0,med_time_horizon,ffae713096867a32e74f633060667153,no,no,no
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,34.150567,stock,2.0,long_time_horizon,ffbda9a14e07718e2b21fb03896d21c5,no,no,no


In [37]:
new_user_data.drop(columns=['churn_by_lastdate','churn_by_consec_diff'],inplace=True)

**Final dataframe**

In [38]:
new_user_data

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,is_churned
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,both,9.701933,stock,100.0,long_time_horizon,0012db34aa7b083f5714e7831195e54d,no
1,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,19.568433,etp,1500.0,long_time_horizon,001d6c77dbdb3213cead7673f250bfdc,no
2,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,22.091867,stock,100.0,long_time_horizon,002e4653171ddc61c3cd30603cd7bd3e,no
3,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,108.555350,stock,500.0,short_time_horizon,00384fa9be6fdca1b786bae70d78f88f,no
4,high_risk_tolerance,good_investment_exp,very_important_liq_need,Android,39.973717,lp,100.0,short_time_horizon,0042aac295a0d4df88f4b83012778bd4,no
...,...,...,...,...,...,...,...,...,...,...
5579,high_risk_tolerance,good_investment_exp,very_important_liq_need,iOS,4.703033,mlp,100.0,short_time_horizon,ff9ee08791e20724a86995ab2bc72578,no
5580,high_risk_tolerance,good_investment_exp,somewhat_important_liq_need,Android,6.988433,stock,250.0,long_time_horizon,ffa12d2f97e310910291f9b26fb2318d,no
5581,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.898000,stock,20.0,med_time_horizon,ffae713096867a32e74f633060667153,no
5582,med_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,34.150567,stock,2.0,long_time_horizon,ffbda9a14e07718e2b21fb03896d21c5,no


**What Percentage of the users have churned in the data provided?**

In [52]:
num_users = len(new_user_data)
num_churned = len(new_user_data[new_user_data['is_churned']=='yes'])

answer1 = 100 * (num_churned/num_users)
answer1

18.0336676217765

**Exploratory data analysis (EDA)**

In [53]:
new_user_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5584 entries, 0 to 5583
Data columns (total 10 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   risk_tolerance                5584 non-null   object 
 1   investment_experience         5584 non-null   object 
 2   liquidity_needs               5584 non-null   object 
 3   platform                      5584 non-null   object 
 4   time_spent                    5584 non-null   float64
 5   instrument_type_first_traded  5584 non-null   object 
 6   first_deposit_amount          5584 non-null   float64
 7   time_horizon                  5584 non-null   object 
 8   user_id                       5584 non-null   object 
 9   is_churned                    5584 non-null   object 
dtypes: float64(2), object(8)
memory usage: 436.4+ KB


In [54]:
new_user_data.isnull().any().sum()

0

In [55]:
new_user_data.duplicated().any().sum()

0

In [50]:
from numpy import unique
for i in range(len(new_user_data.columns)):
    print(new_user_data.columns[i])
    print(unique(new_user_data[new_user_data.columns[i]]))
    print('\n')
               

risk_tolerance
['high_risk_tolerance' 'low_risk_tolerance' 'med_risk_tolerance']


investment_experience
['extensive_investment_exp' 'good_investment_exp' 'limited_investment_exp'
 'no_investment_exp']


liquidity_needs
['not_important_liq_need' 'somewhat_important_liq_need'
 'very_important_liq_need']


platform
['Android' 'both' 'iOS']


time_spent
[0.00000000e+00 9.23166667e-02 1.01450000e-01 ... 1.92282512e+03
 4.03385342e+03 8.78832945e+03]


instrument_type_first_traded
['0' 'adr' 'cef' 'etp' 'lp' 'mlp' 'reit' 'rlt' 'stock' 'tracking' 'wrt']


first_deposit_amount
[0.00000e+00 1.00000e-02 7.00000e-02 1.00000e-01 1.50000e-01 2.60000e-01
 3.50000e-01 4.60000e-01 5.00000e-01 6.00000e-01 1.00000e+00 1.24000e+00
 1.50000e+00 2.00000e+00 2.50000e+00 3.00000e+00 3.10000e+00 3.15000e+00
 3.53000e+00 4.00000e+00 4.43000e+00 5.00000e+00 6.00000e+00 7.00000e+00
 7.05000e+00 7.76000e+00 7.97000e+00 8.00000e+00 8.14000e+00 9.00000e+00
 1.00000e+01 1.02800e+01 1.04900e+01 1.10000e+01 1.10900e+

In [63]:
unique_values = new_user_data.nunique()
unique_values

risk_tolerance                     3
investment_experience              3
liquidity_needs                    3
platform                           3
time_spent                      4502
instrument_type_first_traded      11
first_deposit_amount             317
time_horizon                       3
user_id                         5584
is_churned                         2
dtype: int64

In [69]:
# len(new_user_data[new_user_data['instrument_type_first_traded']=='0'])